In [40]:
import json
import pandas as pd 
import numpy as np
import math
import os
#from elmo import ELMOSequence
from utils import load_data_and_labels, load_glove, filter_embeddings
#from models import ELModel
from preprocessing import ELMoTransformer
from trainer import Trainer

In [54]:
path = r'data/phase2/subcontractor_aug'
label_studio_data = []
for _ in os.listdir(path):
    with open(f'{path}/{_}', 'r', encoding='utf-8') as f:
        lines = json.load(f)
        label_studio_data.append(lines)

In [55]:
#use this for cuad type annotations
dataset = []
j = 0
for lines in label_studio_data:
    for line in lines:
        try:
            context = line['data']['clause']
        except:
            context = line['data']['text']
    # #     print(line)
        annotation = line['annotations']
        payment_terms = []
        all_labels=[]
        for annot in annotation:
            result = annot['result']
            for res in result:
                labels = res['value']['labels']      
                if labels[0] == "What is the language for subcontracting?" or labels[0] == "\"What is the language for subcontracting?\"":
    #                 print(res)
                    value = res['value']
                    start_id = value['start']
                    end_id = value['end']
                    payment_terms.append([start_id, end_id])
                    all_labels.append(labels[0])
#                 elif labels[0] == "What is the ceiling on liquidated damages?" and j<100:
#                     value = res['value']
#                     start_id = value['start']
#                     end_id = value['end']
#                     payment_terms.append([start_id, end_id])
#                     all_labels.append(labels[0])
#                     j+=1 
            if len(payment_terms) > 0:
                dataset.append([payment_terms, context, all_labels])
print(len(dataset))

250


In [ ]:
## with open('data/defect_liability_aug/Defect liability Dutch backtranslation.json', 'r', encoding='utf-8') as f:
#     lines = json.load(f)

In [10]:
# j = 0
# for line in lines:
#     context = line['data']['text']
# # #     print(line)
#     annotation = line['annotations']
#     payment_terms = []
#     all_labels=[]
#     for annot in annotation:
#         result = annot['result']
#         for res in result:
#             labels = res['value']['labels']      
#             if labels[0] == "For how long is the Contractor liable if there are defects?":
# #                 print(res)
#                 value = res['value']
#                 start_id = value['start']
#                 end_id = value['end']
#                 payment_terms.append([start_id, end_id])
#                 all_labels.append(labels[0])
#             elif labels[0] == "If there is failure to rectify, how much of the cost can be claimed from the contractor?" and j<30:
#                 value = res['value']
#                 start_id = value['start']
#                 end_id = value['end']
#                 payment_terms.append([start_id, end_id])
#                 all_labels.append(labels[0])
#                 j+=1 
#         if len(payment_terms) > 0:
#             dataset.append([payment_terms, context, all_labels])
# print(len(dataset))

280


In [4]:
# splitting the json into words and labels. 
complete_data = []
#set of all words in our dataset
vocab = set()

all_context = []
#docanno answers
doccano_answers = []
low = []
words_context = []
x = [] #words in the context
y = [] #B-I-O of each word

#iterate over every data point (sentence - corpus of labelled text) to extract words
for data in dataset: 
    #list to hold all the labels
    labels = []
    #list to hold the extracted answer's class
    extracted_class = []
    #extract the text and annotations from the each data point
    text = data[1]
    indices = data[0]
    label = data[2]
    indices = sorted(indices)
    #print(ord_dict)
    words_context = text.split()             
    #print(words_context) 
    words_inside = []
    words_outside_3 = []
    all_words = []
    words_middle_entities = []
    label_check = []
    
    first = indices[0][0]
    last = indices[len(indices)-1][1]
    outside_entity1 = text[:first]
    words_oe1 = outside_entity1.split()

    outside_entity2 = text[last+1:]
    words_oe2 = outside_entity2.split()

    for i in range(len(indices)):
        start = indices[i][0]
        end = indices[i][1]
        inside_entity = text[start:end+1]
        words_ie = inside_entity.split()
        words_inside.append(words_ie)
#     print(words_inside)
#     print(len(words_inside), words_inside)
#     print(words_inside)
    #the outside entity in between the answers.
    for j in range(len(indices)-1):
        end = indices[j+1][0] 
        start = indices[j][1]
        #print(start,end)
        outside_entity3 = text[start+1:end-1]
        words_oe3 = outside_entity3.split()
        words_outside_3.append(words_oe3)
#     print(len(words_outside_3), words_outside_3)
#     print(words_outside_3)
    label_oe1 = []
    label_oe2 = []
    label_oe3s = []
    label_ies = []
    label_middle_entities = []
    #print(words_inside)
    
    for i in range(len(words_oe1)):
        label_oe1.append('O')
    for i in range(len(words_oe2)):
        label_oe2.append('O')
        
    k = 0  
    for i in range(len(words_inside)):
        label_ie = []
        flag = 0
        for j in range(len(words_inside[i])):
            if label[i] == "What are the performance based liquidated damages?":
                label_ie.append('I')
            else:
                label_ie.append('O')
        label_ies.append(label_ie)
        k+=1
    print(label_ies)
        
    for words_oe3 in words_outside_3:
        label_oe3 = []
        for i in range(len(words_oe3)):
            label_oe3.append('O')
        label_oe3s.append(label_oe3)
#     print(len(label_oe3s), len(label_ies))
    
    for i in range(len(label_oe3s)):
        label_middle_entities += label_ies[i]
        label_middle_entities += label_oe3s[i]
    label_middle_entities += label_ies[-1]
    
    for i in range(len(words_outside_3)):
        words_middle_entities += words_inside[i]
        words_middle_entities += words_outside_3[i]
    words_middle_entities += words_inside[-1]
    
    labels =  label_oe1 + label_middle_entities + label_oe2
    all_words = words_oe1 + words_middle_entities + words_oe2 
    #print(len(labels), len(all_words))
#     print(all_words)
    words_ies = " "
    for i in range(len(words_inside)):
        if 'I' in label_ies[i]:
            words_ies += " " + str(" ".join(words_inside[i]))

    if(len(labels)!=0):
        all_context.append(text)
        complete_data.append((words_context,labels))
        x.append(all_words)
        y.append(labels)
        doccano_answers.append(words_ies)
    else:
        x.append(all_words)
        y.append(['O']*len(words_context))
        doccano_answers.append([])

print(y)

[['I', 'I', 'I', 'I', 'I', 'I', 'I', 'I', 'I', 'I', 'I', 'I', 'I', 'I', 'I', 'I', 'I', 'I', 'I', 'I', 'I', 'I', 'I', 'I', 'I', 'I', 'I', 'I', 'I', 'I', 'I', 'I', 'I', 'I', 'I', 'I', 'I', 'I', 'I', 'I', 'I', 'I', 'I', 'I', 'I', 'I', 'I', 'I', 'I', 'I', 'I', 'I', 'I', 'I', 'I', 'I']]
[['I', 'I', 'I', 'I', 'I', 'I', 'I', 'I', 'I', 'I', 'I', 'I', 'I', 'I', 'I', 'I', 'I']]
[['I', 'I', 'I', 'I', 'I', 'I', 'I', 'I', 'I', 'I', 'I', 'I', 'I', 'I', 'I', 'I', 'I', 'I', 'I', 'I', 'I', 'I', 'I', 'I', 'I', 'I', 'I', 'I', 'I', 'I', 'I', 'I', 'I', 'I', 'I', 'I', 'I', 'I', 'I', 'I', 'I', 'I', 'I', 'I', 'I', 'I', 'I', 'I', 'I', 'I', 'I', 'I', 'I', 'I', 'I', 'I', 'I', 'I', 'I', 'I', 'I', 'I', 'I', 'I', 'I', 'I', 'I', 'I', 'I', 'I', 'I', 'I', 'I', 'I', 'I', 'I', 'I', 'I', 'I', 'I', 'I', 'I', 'I', 'I', 'I', 'I', 'I', 'I', 'I', 'I', 'I', 'I', 'I', 'I', 'I', 'I', 'I', 'I', 'I', 'I', 'I', 'I', 'I', 'I', 'I', 'I', 'I', 'I', 'I', 'I', 'I', 'I', 'I', 'I', 'I', 'I', 'I', 'I', 'I', 'I', 'I', 'I', 'I', 'I', 'I', 'I

[['I', 'I', 'I', 'I', 'I', 'I', 'I', 'I', 'I', 'I', 'I', 'I', 'I', 'I', 'I', 'I', 'I', 'I', 'I', 'I', 'I', 'I', 'I', 'I', 'I', 'I', 'I', 'I', 'I', 'I', 'I', 'I', 'I', 'I', 'I', 'I', 'I', 'I', 'I', 'I', 'I', 'I', 'I', 'I', 'I', 'I', 'I', 'I', 'I', 'I', 'I', 'I', 'I', 'I', 'I', 'I', 'I', 'I', 'I', 'I', 'I', 'I', 'I', 'I', 'I', 'I', 'I', 'I', 'I', 'I']]
[['I', 'I', 'I', 'I', 'I', 'I', 'I', 'I', 'I', 'I', 'I', 'I', 'I', 'I', 'I', 'I', 'I', 'I', 'I', 'I', 'I', 'I', 'I', 'I', 'I', 'I', 'I', 'I', 'I', 'I', 'I', 'I', 'I', 'I', 'I', 'I', 'I', 'I', 'I', 'I', 'I', 'I', 'I', 'I', 'I', 'I', 'I', 'I', 'I', 'I', 'I', 'I', 'I', 'I', 'I', 'I', 'I', 'I', 'I', 'I', 'I', 'I', 'I', 'I', 'I', 'I', 'I', 'I', 'I', 'I', 'I', 'I', 'I', 'I', 'I', 'I', 'I', 'I', 'I', 'I', 'I', 'I', 'I', 'I', 'I', 'I', 'I', 'I', 'I', 'I', 'I', 'I', 'I', 'I', 'I', 'I', 'I', 'I', 'I', 'I', 'I', 'I', 'I', 'I', 'I', 'I', 'I', 'I', 'I', 'I', 'I', 'I', 'I', 'I', 'I', 'I', 'I', 'I', 'I', 'I', 'I', 'I', 'I', 'I', 'I', 'I', 'I', 'I', 'I', 

[['I', 'I', 'I', 'I', 'I', 'I', 'I', 'I', 'I', 'I', 'I', 'I', 'I', 'I', 'I', 'I', 'I', 'I', 'I', 'I', 'I', 'I', 'I', 'I', 'I', 'I', 'I', 'I', 'I', 'I', 'I', 'I', 'I', 'I', 'I', 'I', 'I', 'I', 'I', 'I', 'I', 'I', 'I', 'I', 'I', 'I', 'I', 'I', 'I', 'I', 'I', 'I', 'I', 'I', 'I', 'I', 'I', 'I', 'I', 'I', 'I', 'I', 'I', 'I', 'I', 'I', 'I', 'I', 'I', 'I', 'I', 'I', 'I', 'I', 'I', 'I', 'I', 'I', 'I', 'I', 'I', 'I', 'I', 'I', 'I', 'I', 'I', 'I', 'I', 'I', 'I', 'I', 'I', 'I']]
[['I', 'I', 'I', 'I', 'I', 'I', 'I', 'I', 'I', 'I', 'I', 'I', 'I', 'I', 'I', 'I', 'I', 'I', 'I', 'I', 'I', 'I', 'I', 'I', 'I', 'I', 'I', 'I', 'I', 'I', 'I', 'I', 'I', 'I', 'I', 'I', 'I', 'I', 'I', 'I', 'I', 'I', 'I', 'I', 'I', 'I', 'I', 'I', 'I', 'I', 'I', 'I', 'I', 'I', 'I', 'I', 'I', 'I', 'I', 'I', 'I', 'I', 'I', 'I', 'I', 'I', 'I', 'I', 'I', 'I', 'I', 'I', 'I', 'I', 'I', 'I', 'I', 'I', 'I', 'I', 'I', 'I', 'I', 'I', 'I', 'I', 'I', 'I', 'I', 'I', 'I', 'I', 'I', 'I', 'I', 'I', 'I', 'I', 'I', 'I', 'I', 'I', 'I', 'I', 'I', 

In [5]:
print(x[10])
print(y[10])
print(doccano_answers[10])

['SECTION', '11', '–', 'REMEDIES', '11.1', 'Liquidated', 'Damages', 'for', 'Delay', 'in', 'Final', 'Acceptance', '11.1.1', 'If', 'the', 'Final', 'Acceptance', 'Date', 'does', 'not', 'occur', 'by', 'three', 'days', 'after', 'the', 'Guaranteed', 'Completion', 'Date', '(as', 'the', 'same', 'may', 'be', 'extended', 'in', 'accordance', 'with', 'the', 'terms', 'of', 'this', 'Contract),', 'Contractor', 'shall', 'pay', 'to', 'Company', 'as', 'liquidated', 'damages,', 'for', 'each', 'Day', 'which', 'shall', 'elapse', 'between', 'the', 'Guaranteed', 'Completion', 'Date', 'and', 'the', 'Final', 'Acceptance', 'Date,', 'a', 'sum', 'equal', 'to', '$7000;', 'provided,', 'however,', 'that', 'if', 'the', 'Facility', 'does', 'not', 'achieve', 'Final', 'Acceptance', 'by', 'the', 'Guaranteed', 'Completion', 'Date,', 'but', 'the', 'Facility', 'nevertheless', 'is', 'generating', 'electricity,', 'then', 'the', 'liquidated', 'damages', 'payable', 'by', 'Contractor', 'under', 'this', 'Section', '11.1.1', 'shal

In [5]:
# # splitting the json into words and labels. 
# complete_data = []
# #set of all words in our dataset
# vocab = set()

# all_context = []
# #docanno answers
# doccano_answers = []
# low = []
# words_context = []
# x = []
# y = []

# #iterate over every data point (sentence - corpus of labelled text) to extract words
# for data in dataset: 
#     #list to hold all the labels
#     labels = []
#     #extract the text and annotations from the each data point
#     text = data[1]
#     indices = data[0]
#     #print(ord_dict)
#     words_context = text.split()             
#     #print(words_context) 
#     words_inside = []
#     all_words = []
#     words_middle_entities = []
#     doccano_string = ""
#     start_index = indices[0][0]
#     end_index = indices[0][1]
#     outside_entity1 = text[:start_index]             #outside range 1 (from the beginning of the text to start char)
#     words_oe1 = outside_entity1.split()
#     outside_entity2 = text[end_index+1:]           #outside range 2 (from the end char to the end of the text)
#     words_oe2 = outside_entity2.split()
#     inside_entity = text[start_index:end_index+1]     #inside range (from start char to end char)
#     words_ie = inside_entity.split()
#     label_oe1 = []
#     label_oe2 = []
#     label_oe3s = []
#     label_ies = []
#     label_middle_entities = []
#     #print(words_inside)

#     for i in range(len(words_oe1)):
#         label_oe1.append('O')
#     for i in range(len(words_oe2)):
#         label_oe2.append('O')
#     for i in range(len(words_ie)):
#         if (i==0):
#             label_ies.append('B')
#         else:
#             label_ies.append('I')
#     labels = label_oe1 + label_ies + label_oe2
#     #print(words_context)


#     if(len(labels)!=0):
#         all_context.append(text)
#         complete_data.append((words_context,labels))
#         x.append(words_context)
#         y.append(labels)
#         doccano_answers.append(" ".join(words_ie))
#     else:
#         x.append(words_context)
#         y.append(['O']*len(words_context))
#         doccano_answers.append([])
# # print(complete_data)



In [7]:
print(doccano_answers)
j = 0
for i in range(len(y)):
    if (len(x[i])!=len(y[i])):
        j+=1
        print(i, len(x[i]), len(y[i]))

['  If, upon the expiration of such period, the Net Deliverable Capacity of the Facility as determined by the last Capacity Test is below such Performance Guarantee, then Contractor shall, within thirty (30) Days, pay to Company as liquidated damages, and not as penalty, the amounts set forth in Section 3 of Exhibit K (“Performance Liquidated Damages”);', '  General Contractor agrees to pay Owner the Performance Liquidated Damages (LD) as specified in Exhibit C-3 herein.', '  If no Loans or Senior Debt Commitments in connection therewith are outstanding and the Company or a Restricted Subsidiary has received Performance Liquidated Damages, measured following the Substantial Completion of the last Train to be completed within the Project Facilities contemplated under the EPC Contract (T1/T2), it shall use such Performance Liquidated 74 Damages, within one hundred and eighty (180) days following receipt thereof (or two hundred and seventy (270) days if a commitment to complete, repair, r

In [8]:
to_pop_x = []
for i in range(len(x)):
    if(len(x[i])!=len(y[i])):
        to_pop_x.append(i)
# print(to_pop_x)

for i in range(len(to_pop_x)):
    to_pop_x[i]-=i
# print(to_pop_x)
print(x[0])
print(len(y))

['ARTICLE', '11.', 'PERFORMANCE', 'AND', 'PERFORMANCE', 'RELATED', 'REMEDIES;', 'EXCLUSIVITY;', 'LIMITATION', 'OF', 'LIABILITY', '11.1', 'Performance', 'Liquidated', 'Damages', 'and', 'Performance', 'Bonus', '11.1.1', 'In', 'the', 'event', 'that,', 'upon', 'completion', 'of', 'the', 'Capacity', 'Test,', 'the', 'Net', 'Deliverable', 'Capacity', 'result', 'is', 'less', 'than', 'the', 'Performance', 'Guarantee,', 'Contractor', 'shall', 'have', 'the', 'option', 'for', 'the', 'period', 'specified', 'in', 'Exhibit', 'D', 'to', 'undertake', 'any', 'necessary', 'actions', 'to', 'increase', 'the', 'Net', 'Deliverable', 'Capacity', 'of', 'the', 'Facility', 'to', 'satisfy', 'the', 'Performance', 'Guarantee', 'at', 'its', 'own', 'cost.', 'If,', 'upon', 'the', 'expiration', 'of', 'such', 'period,', 'the', 'Net', 'Deliverable', 'Capacity', 'of', 'the', 'Facility', 'as', 'determined', 'by', 'the', 'last', 'Capacity', 'Test', 'is', 'below', 'such', 'Performance', 'Guarantee,', 'then', 'Contractor', 's

In [13]:
# import pickle
# with open('words_performance_ld_aug.pkl', 'wb') as f:
#     pickle.dump(x, f)
# with open('labels_performance_ld_aug.pkl', 'wb') as g:
#     pickle.dump(y, g)

In [9]:
for i in to_pop_x:
    x.pop(i)
    y.pop(i)

In [10]:
ratio_tt = math.floor(len(x)*0.8)  
x_train, x_test = x[:ratio_tt], x[ratio_tt:]
y_train, y_test = y[:ratio_tt], y[ratio_tt:]
print(x_train[0])
print(y_train[0])

['ARTICLE', '11.', 'PERFORMANCE', 'AND', 'PERFORMANCE', 'RELATED', 'REMEDIES;', 'EXCLUSIVITY;', 'LIMITATION', 'OF', 'LIABILITY', '11.1', 'Performance', 'Liquidated', 'Damages', 'and', 'Performance', 'Bonus', '11.1.1', 'In', 'the', 'event', 'that,', 'upon', 'completion', 'of', 'the', 'Capacity', 'Test,', 'the', 'Net', 'Deliverable', 'Capacity', 'result', 'is', 'less', 'than', 'the', 'Performance', 'Guarantee,', 'Contractor', 'shall', 'have', 'the', 'option', 'for', 'the', 'period', 'specified', 'in', 'Exhibit', 'D', 'to', 'undertake', 'any', 'necessary', 'actions', 'to', 'increase', 'the', 'Net', 'Deliverable', 'Capacity', 'of', 'the', 'Facility', 'to', 'satisfy', 'the', 'Performance', 'Guarantee', 'at', 'its', 'own', 'cost.', 'If,', 'upon', 'the', 'expiration', 'of', 'such', 'period,', 'the', 'Net', 'Deliverable', 'Capacity', 'of', 'the', 'Facility', 'as', 'determined', 'by', 'the', 'last', 'Capacity', 'Test', 'is', 'below', 'such', 'Performance', 'Guarantee,', 'then', 'Contractor', 's

In [11]:
from anago import wrapper
import anago
# from elmo import ELMOSequence
model = anago.Sequence(word_embedding_dim=100, char_embedding_dim=50, word_lstm_size=100, char_lstm_size=50, fc_dim=50, dropout=0.15)
#model.fit(x_train, y_train, epochs=30, batch_size=8)

In [99]:
model.save('price_escalation/restrictions_on_pe_weights_v1','price_escalation/restrictions_on_pe_params_v1','price_escalation/restrictions_on_pe_preprocessor_v1')

In [12]:
model_new = model.load('liquid_damages/performance_ld_aug/performance_ld_aug_weights_2','liquid_damages/performance_ld_aug/performance_ld_aug_params_2','liquid_damages/performance_ld_aug/performance_ld_aug_preprocessor_2')

In [13]:
print(model_new)

In [16]:
print(all_context[0])

ARTICLE 11.      PERFORMANCE AND PERFORMANCE RELATED REMEDIES; EXCLUSIVITY; LIMITATION OF LIABILITY           11.1      Performance Liquidated Damages and Performance Bonus                        11.1.1      In the event that, upon completion of the Capacity Test, the Net Deliverable Capacity result is less than the Performance Guarantee, Contractor shall have the option for the period specified in Exhibit D to undertake any necessary actions to increase the Net Deliverable Capacity of the Facility to satisfy the Performance Guarantee at its own cost. If, upon the expiration of such period, the Net Deliverable Capacity of the Facility as determined by the last Capacity Test is below such Performance Guarantee, then Contractor shall, within thirty (30) Days, pay to Company as liquidated damages, and not as penalty, the amounts set forth in Section 3 of Exhibit K (“Performance Liquidated Damages”); provided, however, that if Contractor has made any payments of Shortfall Generation Liquid

In [17]:
answer_payment_terms = []
i = 0
for context in all_context:
    i+=1
    print(i)
    if(len(model_new.analyze(context)['entities'])!=0):
        ans = model_new.analyze(context)['entities']
        answer_payment_terms.append(ans)
    else:
        answer_payment_terms.append([])
    

1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
277


In [18]:
#import modules
import string,re

In [19]:
# these functions are heavily influenced by the HF squad_metrics.py script
def normalize_text(s):
    """Removing articles and punctuation, and standardizing whitespace are all typical text processing steps."""
    import string, re

    def remove_articles(text):
        regex = re.compile(r"\b(a|an|the)\b", re.UNICODE)
        return re.sub(regex, " ", text)

    def white_space_fix(text):
        return " ".join(text.split())

    def remove_punc(text):
        exclude = set(string.punctuation)
        return "".join(ch for ch in text if ch not in exclude)

    def lower(text):
        return text.lower()

    return white_space_fix(remove_articles(remove_punc(s)))

In [20]:
#to compute the f1 score

def compute_f1(prediction, truth):
    pred_tokens = normalize_text(prediction).split()
    truth_tokens = normalize_text(truth).split()
    
    # if either the prediction or the truth is no-answer then f1 = 1 if they agree, 0 otherwise
    if len(pred_tokens) == 0 or len(truth_tokens) == 0:
        return int(pred_tokens == truth_tokens)
    
    common_tokens = set(pred_tokens) & set(truth_tokens)
    
    # if there are no common tokens then f1 = 0
    
    prec = len(common_tokens) / len(pred_tokens)
    rec = len(common_tokens) / len(truth_tokens)

    if prec!=0 and rec!=0:
        return 2 * (prec * rec) / (prec + rec)
    else:
        return 0

In [21]:
f1_subscore = []
f1_score = []
model_answers = []
# for (answer,cor) in zip(predicted_answers,correct_answers):
#     print(f1_scoring(str(answer),str(cor)))
for ans in answer_payment_terms:
    sub_answers = []
    for j in range(len(ans)):
        sub_answers.append(ans[j]['text'])
    model_answers.append(" ".join(sub_answers))


In [22]:
print(answer_payment_terms[2])
print(doccano_answers[2])

[{'text': '(a) If no Loans or Senior Debt Commitments in connection therewith are outstanding and the Company or a Restricted Subsidiary has received Performance Liquidated Damages, measured following the Substantial Completion of the last Train to be completed within the Project Facilities contemplated under the EPC Contract (T1/T2), it shall use such Performance Liquidated 74 Damages, within one hundred and eighty (180) days following receipt thereof (or two hundred and seventy (270) days if a commitment to complete, repair, refurbish or improve the Project Facilities is entered within one hundred and eighty (180) days following the receipt of such proceeds) to: (1) complete, repair, refurbish or improve the Project Facilities in respect of which the Performance Liquidated Damages were paid or other Project Facilities under construction related to the Corpus Christi Terminal Facility or the Corpus Christi Pipeline; or (2) repay or reimburse providers of Equity Funding to the extent s

In [23]:
f1_score = []
for i in range(len(doccano_answers)):
    f1_score.append(compute_f1(model_answers[i], doccano_answers[i]))

In [24]:
count = 0
for score in f1_score:
    if score == 0:
        count +=1 
print(count)

25


In [25]:
print(len(doccano_answers))
print(len(model_answers))
print(answer_payment_terms[1])

281
281
[]


In [56]:
# best_pred_ans = []
# for i in range(len(f1_score)):
#     print(f1_score[i])
#     if(len(f1_score[i])!=0):
#         idx = f1_score[i].index(max(f1_score[i]))
#         print(idx)
#         best_pred_ans.append(model_answers[i][idx])
#     else:
#         best_pred_ans.append([])

In [57]:
# best_f1 = []
# for i in range(len(best_pred_ans)):
#     best_f1.append(compute_f1(best_pred_ans[i], doccano_answers[i]))

In [26]:
import pandas as pd 
data ={'Context' : all_context, 'Actual Answer':  doccano_answers, 'Predicted Answer': model_answers, 'F1': f1_score}
df = pd.DataFrame(data)

In [27]:
df.tail(50)

,Context,Actual Answer,Predicted Answer,F1
231,2.05 Advance Payment. (a) Optional. Borrower ...,If the Project Company receives performance ...,prepay the loan in an amount equal to the amou...,0.202899
232,8. Performance Liquidation Damages Contractor...,1% monthly fee will be charged if any of the...,for City-initiated contractual payment losses ...,0.500000
233,Article IV Lease Commencement Section 4.1 Not...,After Company has provided the notice requir...,but not later than 30 days prior to the schedu...,0.515152
234,38. Indemnity: Seller agrees to pay Buyer the...,The EPC contractor has guaranteed a net powe...,EPC contractor has guaranteed a net power outp...,0.391608
235,4.9 Performance Tests and Performance Liquida...,If the Facility PR measured during the compl...,,0.000000
236,12. Performance Liquidation Damages. The part...,The parties agree to the following performan...,parties agree to the following performance liq...,0.586207
237,Article 6 Delay; performance during testing; ...,Contractor will pay performance liquidation ...,,0.000000
238,ARTICLE 27 - STATUTORY DAMAGES 27.1 Owner Dam...,Performance contract damages are established...,damages are established only for guaranteed ne...,0.434783
239,10 Warranty 10.1 Service Warranty. The O&amp;...,If the measured PerformanceAvailability of t...,If the measured PerformanceAvailability of the...,0.450704
240,2. OPERATOR&#39;S GENERAL RESPONSIBILITIES 2....,the Operator&#39;s failure to comply with an...,the service for the month in which Employer th...,0.652406


In [52]:
df.to_csv('performance_ld_aug_results.csv')

In [28]:
print(sum(f1_score)/len(f1_score))

0.5416736899084851


In [13]:
!pip install fuzzywuzzy

In [29]:
from nltk.tokenize import sent_tokenize
import nltk
from fuzzywuzzy import fuzz

#postprocessing for liability period

processed_answers = []
for i in range(len(model_answers)):
    sents_context = sent_tokenize(all_context[i])
    sents_ans = sent_tokenize(model_answers[i])
    model_ans_tokens = [nltk.word_tokenize(sent) for sent in sents_ans]
    context_tokens = [nltk.word_tokenize(sent) for sent in sents_context]
    completed_sentences = []
    for model_ans_token in model_ans_tokens:
        for context_token in context_tokens:
            if set(model_ans_token).issubset(set(context_token)):
                completed_sent = ' '.join(context_token)
                completed_sent = re.sub(r'\s*([,.?!;:&#])\s*', r'\1', completed_sent)
                pattern_bracket = r'\(\s*(.*?)\s*\)'
                new_ans = re.sub(pattern_bracket, lambda m: '(' + m.group(1).replace(' ', '') + ')', completed_sent)
                completed_sentences.append(new_ans)
                break
    processed_answers.append(" ".join(completed_sentences))
            
print(len(processed_answers))
        

C:\Users\Hp\anaconda3\envs\anago\lib\site-packages\fuzzywuzzy\fuzz.py:11: UserWarning: Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning
  warnings.warn('Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning')


281


In [30]:
new_processed_answers = []
for ans in processed_answers:
    print(ans)
#     pattern_bracket = r'\(\s*(.*?)\s*\)'
#     new_ans = re.sub(pattern, lambda m: '(' + m.group(1).replace(' ', '') + ')', ans)
#     print(new_ans)

PERFORMANCE AND PERFORMANCE RELATED REMEDIES;EXCLUSIVITY;LIMITATION OF LIABILITY 11.1 Performance Liquidated Damages and Performance Bonus 11.1.1 In the event that,upon completion of the Capacity Test,the Net Deliverable Capacity result is less than the Performance Guarantee,Contractor shall have the option for the period specified in Exhibit D to undertake any necessary actions to increase the Net Deliverable Capacity of the Facility to satisfy the Performance Guarantee at its own cost. If,upon the expiration of such period,the Net Deliverable Capacity of the Facility as determined by the last Capacity Test is below such Performance Guarantee,then Contractor shall,within thirty (30) Days,pay to Company as liquidated damages,and not as penalty,the amounts set forth in Section 3 of Exhibit K (“PerformanceLiquidatedDamages”);provided,however,that if Contractor has made any payments of Shortfall Generation Liquidated Damages to Company pursuant to Section 4.10,Contractor shall be obligate

In [32]:
# f1_score = []
# for i in range(len(doccano_answers)):
#     f1 = compute_f1(processed_answers[i], doccano_answers[i])
#     print(processed_answers[i])
#     print(doccano_answers[i])
#     print(f1)
#     f1_score.append(f1)

PERFORMANCE AND PERFORMANCE RELATED REMEDIES;EXCLUSIVITY;LIMITATION OF LIABILITY 11.1 Performance Liquidated Damages and Performance Bonus 11.1.1 In the event that,upon completion of the Capacity Test,the Net Deliverable Capacity result is less than the Performance Guarantee,Contractor shall have the option for the period specified in Exhibit D to undertake any necessary actions to increase the Net Deliverable Capacity of the Facility to satisfy the Performance Guarantee at its own cost. If,upon the expiration of such period,the Net Deliverable Capacity of the Facility as determined by the last Capacity Test is below such Performance Guarantee,then Contractor shall,within thirty (30) Days,pay to Company as liquidated damages,and not as penalty,the amounts set forth in Section 3 of Exhibit K (“PerformanceLiquidatedDamages”);provided,however,that if Contractor has made any payments of Shortfall Generation Liquidated Damages to Company pursuant to Section 4.10,Contractor shall be obligate

In [40]:
from sklearn.metrics import f1_score
f1_scores = []
for i in range(len(doccano_answers)):
    model_set = set(model_answers[i])
    #print(model_set)
    actual_set = set(doccano_answers[i])
    #print(actual_set)
    #     print(model_set)
    tp = len(model_set.intersection(actual_set))
    fp = len(model_set) - tp
    fn = len(actual_set) - tp

    if tp == 0 and (fp != 0 or fn != 0):
        # Handle the case where both precision and recall are zero
        f1 = 0.0
    else:
        precision = tp / (tp + fp)
        recall = tp / (tp + fn)
        f1 = 2 * (precision * recall) / (precision + recall)
    f1_scores.append(f1)

In [41]:
print(sum(f1_scores)/len(f1_scores))

0.8517572084432007


In [42]:
import pandas as pd 
data ={'Context' : all_context, 'Actual Answer':  doccano_answers, 'Predicted Answer': model_answers, 'F1': f1_scores}
df = pd.DataFrame(data)

In [44]:
df.head(20)

,Context,Actual Answer,Predicted Answer,F1
0,ARTICLE 11. PERFORMANCE AND PERFORMANCE R...,"If, upon the expiration of such period, the ...",to satisfy the Performance Guarantee at its ow...,0.988764
1,ARTICLE XIII SCHEDULE AND PERFORMANCE GUARANTE...,General Contractor agrees to pay Owner the P...,,0.000000
2,Section 4.14 Liens. Subject to Section 3 of...,If no Loans or Senior Debt Commitments in co...,(a) If no Loans or Senior Debt Commitments in ...,0.989474
3,"ARTICLE 8 PERFORMANCE TESTS, GUARANTEES & LIQU...",Owner may assess the following Performance L...,Owner may assess the following Performance Liq...,0.800000
4,ARTICLE 10 WARRANTY 10.1 Services Warranty. O&...,If the Measured Availability for the PV Plan...,If the Measured Availability for the PV Plant ...,0.990099
5,"ARTICLE 11 COMPLETION 11.1Notice of RFSU, Deli...",If Subproject 3 has not achieved the Perform...,,0.000000
6,16. LIQUIDATED DAMAGES; EARLY COMPLETION BO...,"Contractor agrees that if, based on the resu...","Contractor agrees that if, based on the result...",0.989247
7,ARTICLE 20 LIMITATION OF LIABILITY 20.1 Contra...,"Subject to Section 20.2C, Contractor’s maxim...","to Section 20.2C, Contractor’s maximum liabili...",1.000000
8,ARTICLE 6 DELAY; TESTING; PERFORMANC...,If the Facility fails to achieve any or all ...,If the Facility fails to achieve any or all of...,0.977273
9,10.5 Performance Tests. 10.5.1 Conduct of Perf...,"Notwithstanding anything to the contrary, Pe...","Notwithstanding anything to the contrary, Perf...",1.000000


In [43]:
df.to_csv('performance_ld_aug_results.csv')